In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [ ]:
# nifty_50_componants_data_from_tradingview.csv
nifty_comprehensive_df = pd.read_csv("D:/Programming/Download_Backtest_Deploy_data/1__Download/1__Download_data_ICICI_via_API/nifty_50_componants_data_from_tradingview.csv")
nifty_comprehensive_df.head(2)

In [ ]:
# nifty_50_componants_data_from_tradingview.csv
corporate_action_df = pd.read_csv(r"D:\Programming\Download_Backtest_Deploy_data\1__Download\CF_CA_equities_01_01_2021_to_23_09_2025.csv")
corporate_action_df.head(2)

In [ ]:
# Filter corporate_action_df to only include companies common in both datasets
companies_in_both = set(nifty_comprehensive_df['Symbol'].unique())
corporate_action_filtered = corporate_action_df[corporate_action_df['SYMBOL'].isin(companies_in_both)].reset_index(drop=True)
corporate_action_filtered

In [ ]:
import pandas as pd

# Assume df is your DataFrame loaded from the NSE CSV
# Columns: SYMBOL, COMPANY NAME, PURPOSE, FACE VALUE, EX-DATE, etc.

# Define keywords that indicate volume-affecting corporate actions
volume_keywords = [
    "BONUS",
    "SPLIT",
    "SUB-DIVISION",
    "CONSOLIDATION",
    "MERGER",
    "AMALGAMATION",
    "SCHEME",
    "RIGHTS"
]

# Filter PURPOSE column by these keywords (case-insensitive)
mask = corporate_action_filtered["PURPOSE"].str.upper().str.contains("|".join(volume_keywords), na=False)

df_volume_affecting = corporate_action_filtered[mask].copy()

# Show first few filtered rows
aa = df_volume_affecting[["SYMBOL", "PURPOSE", "EX-DATE"]].reset_index(drop=True)
aa.to_csv("D:/Programming/corporate_actions_affecting_volume.csv", index=False)

df_volume_affecting

In [ ]:
import json
from datetime import datetime
import re

# Load the current nifty50_data.json file
json_file_path = r"D:\Programming\Download_Backtest_Deploy_data\1__Download\TradingView_data_download\nifty50_data.json"

with open(json_file_path, 'r') as f:
    nifty50_data = json.load(f)

# Function to parse purpose and extract action type and ratio
def parse_purpose(purpose):
    purpose_lower = purpose.lower()
    
    if 'bonus' in purpose_lower:
        # Extract ratio from bonus
        if '1:1' in purpose:
            return 'bonus', '1:1'
        elif '2:1' in purpose:
            return 'bonus', '2:1'
        elif '4:1' in purpose:
            return 'bonus', '4:1'
        elif '1:3' in purpose:
            return 'bonus', '1:3'
        else:
            return 'bonus', ''
    
    elif 'rights' in purpose_lower:
        # Extract ratio and price from rights
        if '1:14' in purpose:
            return 'rights', '1:14'
        elif '6:179' in purpose:
            return 'rights', '6:179'
        elif '1:26' in purpose:
            return 'rights', '1:26'
        else:
            return 'rights', ''
    
    elif 'split' in purpose_lower or 'sub-division' in purpose_lower:
        # Extract clean stock split ratio
        ratio = extract_stock_split_ratio(purpose)
        return 'stock_split', ratio
    
    elif 'demerger' in purpose_lower:
        return 'demerger', ''
    
    elif 'dividend' in purpose_lower:
        return 'dividend', purpose.split()[-1] if len(purpose.split()) > 1 else ''
    
    else:
        return 'corporate_action', ''

# Function to extract clean stock split ratio from messy text
def extract_stock_split_ratio(purpose_text):
    """
    Convert text like "From Rs 10/- Per Share To Rs 1/- Per Share" 
    to clean ratio like "10:1"
    """
    try:
        # Pattern to match "From Rs X/- Per Share To Rs Y/- Per Share"
        pattern = r'from\s+rs\s+(\d+)/-.*?to\s+rs\s+(\d+)/-'
        match = re.search(pattern, purpose_text.lower())
        
        if match:
            from_value = int(match.group(1))
            to_value = int(match.group(2))
            
            # Stock split ratio is from_value:to_value
            return f"{from_value}:{to_value}"
        
        # Alternative pattern for different formats
        # Pattern to match "Rs 2/- to Rs 1/-"
        alt_pattern = r'rs\s+(\d+)/-.*?rs\s+(\d+)/-'
        alt_match = re.search(alt_pattern, purpose_text.lower())
        
        if alt_match:
            from_value = int(alt_match.group(1))
            to_value = int(alt_match.group(2))
            return f"{from_value}:{to_value}"
            
        # If no pattern matches, return the original text cleaned up
        return purpose_text.split(' - ')[-1] if ' - ' in purpose_text else purpose_text
        
    except Exception as e:
        # Fallback to original method if regex fails
        return purpose_text.split(' - ')[-1] if ' - ' in purpose_text else purpose_text

# Function to convert date format from DD-MMM-YYYY to YYYY-MM-DD
def convert_date_format(date_str):
    try:
        # Parse the date
        date_obj = datetime.strptime(date_str, "%d-%b-%Y")
        return date_obj.strftime("%Y-%m-%d")
    except:
        return date_str

# Create new corporate actions list from df_volume_affecting
new_corporate_actions = []

for idx, row in df_volume_affecting.iterrows():
    action_type, ratio = parse_purpose(row['PURPOSE'])
    
    corporate_action = {
        "symbol": row['SYMBOL'],
        "action_type": action_type,
        "ratio": ratio,
        "effective_date": convert_date_format(row['EX-DATE'])
    }
    
    # Add price field for rights issues
    if action_type == 'rights' and 'Rs' in row['PURPOSE']:
        price_part = row['PURPOSE'].split('Rs')[-1].strip()
        corporate_action["price"] = f"Rs {price_part}"
    
    new_corporate_actions.append(corporate_action)

# Update the corporate_actions in nifty50_data
nifty50_data['corporate_actions'] = new_corporate_actions

# Save the updated JSON file
with open(json_file_path, 'w') as f:
    json.dump(nifty50_data, f, indent=4)

print("Successfully updated nifty50_data.json with corporate actions data!")
print(f"Total corporate actions added: {len(new_corporate_actions)}")

# Display sample of the formatted ratios for verification
print("\nSample of formatted stock split ratios:")
for action in new_corporate_actions:
    if action['action_type'] == 'stock_split':
        print(f"{action['symbol']}: {action['ratio']}")